In [80]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
import urllib.request
import os
from pyquery import PyQuery as pq
import pickle
from selenium import webdriver
import datetime as dt
import time
import calendar
import scipy.stats as ss
from pandas import Series
import math
import matplotlib.pyplot as plt

In [2]:
#整理資料
origin_list=pd.read_csv("Materials Equity ETF List (62).csv",index_col=22,parse_dates=True)
origin_list2=pd.read_csv("Telecom Equity ETF List (10).csv",index_col=22,parse_dates=True)
need_list=pd.concat([origin_list,origin_list2])

In [3]:
#切出symbol欄&ETF名稱欄
symbol_list = need_list['Symbol'].reset_index()['Symbol']
name_list = need_list['ETF Name'].reset_index()['ETF Name']
all_ETF=need_list["Symbol"].values

In [4]:
all_ETF

array(['XLB', 'IYM', 'IGE', 'VAW', 'PKB', 'XHB', 'ITB', 'GDX', 'XME',
       'MXI', 'PYZ', 'SLX', 'RTM', 'UYM', 'SMN', 'FXZ', 'MOO', 'CUT',
       'WOOD', 'HAP', 'PSAU', 'PAGG', 'FLM', 'GRES', 'GDXJ', 'CHIM',
       'FTRI', 'FTAG', 'SBM', 'PSCM', 'SIL', 'COPX', 'LIT', 'GNR', 'REMX',
       'GOEX', 'URA', 'NUGT', 'DUST', 'CROP', 'SOIL', 'GUNR', 'PICK',
       'RING', 'SLVP', 'VEGI', 'SILJ', 'JNUG', 'JDST', 'FMAT', 'SGDM',
       'GDXX', 'GDXS', 'HOML', 'SGDJ', 'NAIL', 'NANR', 'VOX', 'ERUS',
       'IYZ', 'FCOM', 'IXP', 'XTL', 'WBIF', 'FONE', 'LTL'], dtype=object)

In [5]:
#combine ETF daily data
#use the combined data to build the weekly and monthly ETF return 
def combine_daily_data(ETF):
    com_data=pd.DataFrame()
    for s in range(len(ETF)):
        read = "./data/"+ ETF[s] +".csv"
        data = pd.read_csv(read,index_col=0,parse_dates=True)["Adj Close"]
        com_data = pd.concat([com_data,data],axis=1)
    com_data.columns = ETF
    return com_data

Daily_ETF = combine_daily_data(all_ETF)

In [59]:
#fill in the no-trade dates of ETF
day=Daily_ETF.index[0]
n=0
while day!=Daily_ETF.index[-1]:
    if (Daily_ETF.index[n+1]-Daily_ETF.index[n]).days !=1:
        for i in range(1,(Daily_ETF.index[n+1]-Daily_ETF.index[n]).days):
            Daily_ETF.loc[Daily_ETF.index[n]+dt.timedelta(i)]=Daily_ETF.loc[Daily_ETF.index[n]]
    Daily_ETF=Daily_ETF.sort_index()
    n = n+(Daily_ETF.index[n+1]-Daily_ETF.index[n]).days
    day=Daily_ETF.index[n]

In [60]:
# week index is to select every Friday and month index is the first day of every month
week_index=[]
for i in range(Daily_ETF.shape[0]):
    if Daily_ETF.index[i].isoweekday()==5:
        week_index.append(Daily_ETF.index[i])
month_index=[]
for i in range(Daily_ETF.shape[0]):
    if Daily_ETF.index[i].day==1:
        month_index.append(Daily_ETF.index[i])

In [8]:
#fill in the no-trade dates of 3 month US bond (using its value as risk-free interest rate) 
riskless_rate=pd.read_csv("USG3M.csv",index_col=0,parse_dates=True, )
USG3M=riskless_rate["Price"].reset_index("Date")
USG3M.set_index("Date" , inplace=True)
USG3M=USG3M[::-1]
n=0
day=USG3M.index[0]
while day!=USG3M.index[-1]:
    if (USG3M.index[n+1]-USG3M.index[n]).days !=1:
        for i in range(1,(USG3M.index[n+1]-USG3M.index[n]).days):
            USG3M.loc[USG3M.index[n]+dt.timedelta(i)]=USG3M.loc[USG3M.index[n]]
    USG3M=USG3M.sort_index()
    n = n+(USG3M.index[n+1]-USG3M.index[n]).days
    day=USG3M.index[n]

In [71]:
weekly_ETF=Daily_ETF.loc[week_index]
monthly_ETF=Daily_ETF.loc[month_index]
wr=weekly_ETF/weekly_ETF.shift(1)-1
mr=monthly_ETF/monthly_ETF.shift(1)-1
wUSG3M = USG3M.loc[week_index] / 100 / 52
mUSG3M = USG3M.loc[month_index] / 100 / 12

In [72]:
#Portfolio performance evaluation with generalized Sharpe ratios:
#Beyond the mean and variance
#For weekly data
#3 years for a round

# potential bug r_f , negative number in the sqrt

wASKSR = []

for now, symbol in enumerate(symbol_list):
    wASKSR.append([])
    print("computing weekly ETF's ASKSR : ", symbol)
    List = wr[symbol]
    #print(List.shape[0]) #length of the series
    l = len(List)
    for i in range(l-52*3):
        if(not ( float("-inf") <List[i] < float("inf") )): #nan
            wASKSR[now].append(float('nan'))
            continue
        else:
            X = []
            for j in range(i, i+52*3):
                X.append(List[j])
            X = Series(X)
            #computing ASKSR
            K = X.kurt() #Kurt
            S = X.skew() #Skew
            std = X.std() #standard deviation

            #compute alpha
            tmp = 3*K-4*S**2
            if tmp < 0 :
                tmp *= -1
            alpha = 3*math.sqrt(tmp) / (std**2 * (3*K - 5 * S**2))

            #compute beta
            beta =  3 * S / (std*(3*K-5*S**2))

            #compute eta (n with right tail)
            eta = X.mean() - 3*S*std/(3*K-4*S**2)

            #compute delta
            tmp = 3*K-5*S**2
            if tmp < 0:
                tmp *= -1
            delta = 3 * std * math.sqrt(tmp)/(3*K - 4*S**2)

            r_f = wUSG3M["Price"][i+52*3] # risk-free interest rate per unit of time

            # a* x lambda 
            a_star_multiple_lambda =  (beta + alpha*(eta - r_f) / math.sqrt(delta**2 + (eta - r_f)**2) )
            #print(a_star_multiple_lambda)

            #compute psi
            tmp = alpha**2 - beta**2
            if tmp < 0:
                tmp *= -1
            psi = math.sqrt(tmp)

            #equation 38
            tmp = alpha**2-(beta-a_star_multiple_lambda)**2
            if tmp < 0 :
                tmp *= -1
            tmp = 2*(a_star_multiple_lambda*(eta - r_f) - delta*(psi - math.sqrt(tmp)))
            if tmp < 0:
                tmp *= -1 
            ASKSR = math.sqrt(tmp)
            wASKSR[now].append(ASKSR)
    if(len(wASKSR[now])!= 105):
        print(len(wASKSR[now]))
#print(wASKSR)

computing weekly ETF's ASKSR :  XLB
computing weekly ETF's ASKSR :  IYM
computing weekly ETF's ASKSR :  IGE
computing weekly ETF's ASKSR :  VAW
computing weekly ETF's ASKSR :  PKB
computing weekly ETF's ASKSR :  XHB
computing weekly ETF's ASKSR :  ITB
computing weekly ETF's ASKSR :  GDX
computing weekly ETF's ASKSR :  XME
computing weekly ETF's ASKSR :  MXI
computing weekly ETF's ASKSR :  PYZ
computing weekly ETF's ASKSR :  SLX
computing weekly ETF's ASKSR :  RTM
computing weekly ETF's ASKSR :  UYM
computing weekly ETF's ASKSR :  SMN
computing weekly ETF's ASKSR :  FXZ
computing weekly ETF's ASKSR :  MOO
computing weekly ETF's ASKSR :  CUT
computing weekly ETF's ASKSR :  WOOD
computing weekly ETF's ASKSR :  HAP
computing weekly ETF's ASKSR :  PSAU
computing weekly ETF's ASKSR :  PAGG
computing weekly ETF's ASKSR :  FLM
computing weekly ETF's ASKSR :  GRES
computing weekly ETF's ASKSR :  GDXJ
computing weekly ETF's ASKSR :  CHIM
computing weekly ETF's ASKSR :  FTRI
computing weekly ETF'

In [82]:
#Portfolio performance evaluation with generalized Sharpe ratios:
#Beyond the mean and variance
#For monthly data
#3 years for a round 
mASKSR = []
for now, symbol in enumerate(symbol_list):
    mASKSR.append([])
    print("computing monthly ETF's ASKSR : ",symbol)
    List = mr[symbol]
    #print(List.shape[0]) #length of the series
    l = List.shape[0]
    for i in range(l - 12*3):
        if(not ( float("-inf") <List[i] < float("inf") )): #nan
            mASKSR[now].append(float('nan'))
            continue
        X = []
        for j in range(i, i+12*3):
            X.append(List[j])
        X = Series(X)
        #computing ASKSR
        K = X.kurt() #Kurt
        S = X.skew() #Skew
        std = X.std() #stdev
        
        #compute alpha
        tmp = 3*K-4*S**2
        if tmp < 0 :
            tmp *= -1
        alpha = 3*math.sqrt(tmp) / (std**2 * (3*K - 5 * S**2))
        
        #compute beta
        beta =  3 * S / (std*(3*K-5*S**2))
        
        #compute eta (n with right tail)
        eta = X.mean() - 3*S*std/(3*K-4*S**2)
        
        #compute delta
        tmp = 3*K-5*S**2
        if tmp < 0:
            tmp *= -1
        delta = 3 * std * math.sqrt(tmp)/(3*K - 4*S**2)
        
        r_f = mUSG3M["Price"][i+12*3] # risk-free interest rate per unit of time

        # a* x lembda
        a_star_multiple_lambda =  (beta + alpha*(eta - r_f) / math.sqrt(delta**2 + (eta - r_f)**2) )
        
        #compute psi
        tmp = alpha**2 - beta**2
        if tmp < 0:
            tmp *= -1
        psi = math.sqrt(tmp)
        #print(psi)
        
        # equation 38
        tmp = alpha**2-(beta-a_star_multiple_lambda)**2
        if tmp < 0 :
            tmp *= -1
        tmp = 2*(a_star_multiple_lambda*(eta - r_f) - delta*(psi - math.sqrt(tmp)))
        if tmp < 0:
            tmp *= -1 
        ASKSR = math.sqrt(tmp)
        #print("TIME:",List.index[i+52*3],"ASKSR:", ASKSR)
        mASKSR[now].append(ASKSR)
print("monthly ASKSR done")

computing monthly ETF's ASKSR :  XLB
computing monthly ETF's ASKSR :  IYM
computing monthly ETF's ASKSR :  IGE
computing monthly ETF's ASKSR :  VAW
computing monthly ETF's ASKSR :  PKB
computing monthly ETF's ASKSR :  XHB
computing monthly ETF's ASKSR :  ITB
computing monthly ETF's ASKSR :  GDX
computing monthly ETF's ASKSR :  XME
computing monthly ETF's ASKSR :  MXI
computing monthly ETF's ASKSR :  PYZ
computing monthly ETF's ASKSR :  SLX
computing monthly ETF's ASKSR :  RTM
computing monthly ETF's ASKSR :  UYM
computing monthly ETF's ASKSR :  SMN
computing monthly ETF's ASKSR :  FXZ
computing monthly ETF's ASKSR :  MOO
computing monthly ETF's ASKSR :  CUT
computing monthly ETF's ASKSR :  WOOD
computing monthly ETF's ASKSR :  HAP
computing monthly ETF's ASKSR :  PSAU
computing monthly ETF's ASKSR :  PAGG
computing monthly ETF's ASKSR :  FLM
computing monthly ETF's ASKSR :  GRES
computing monthly ETF's ASKSR :  GDXJ
computing monthly ETF's ASKSR :  CHIM
computing monthly ETF's ASKSR : 

In [84]:
#2011 JBF omega performance messure ...
#we need Sharpe_omega(L) which equals to omega_x(L) - 1
#F(.) is the cdf of the asset return X defined on the interval (a,b)
#and omega_x(L) = ( integrate from L to b : (1-F(x))dx) / ( integrate from a to L : F(x)dx )
#where L is the return threshold selected by the investor 
#for weekly:
wOmega = []
for now, symbol in enumerate(symbol_list):
    wOmega.append([])
    print("Computing weekly Omega of ETF : ", symbol)
    List = wr[symbol]
    #print(List.shape[0]) #length of the series
    l = List.shape[0]
    for i in range(l-52*3):
            
        L =  wUSG3M["Price"][i+52*3] #risk-free interest rate 
        
        if(not ( float("-inf") <List[i] < float("inf") )): #nan
            wOmega[now].append(float('nan'))
            continue
        X = []
        for j in range(i, i+52*3):
            X.append(List[j])
        X.sort()
        a = X[0]
        b = X[len(X)-1]
        # using 0.001 as a minimum interval 
        left_part = 0
        right_part = 0
        for i in range(len(X)):
            if(X[i] < L):
                left_part +=  L - X[i]
            else :
                right_part += X[i] - L
#        print(right_part, left_part)
        omega = right_part / left_part
        wOmega[now].append(omega)
    if(len(wOmega[now])!= 105):
        print(len(wOmega[now]))
print("weekly omega done")

Computing weekly Omega of ETF :  XLB
Computing weekly Omega of ETF :  IYM
Computing weekly Omega of ETF :  IGE
Computing weekly Omega of ETF :  VAW
Computing weekly Omega of ETF :  PKB
Computing weekly Omega of ETF :  XHB
Computing weekly Omega of ETF :  ITB
Computing weekly Omega of ETF :  GDX
Computing weekly Omega of ETF :  XME
Computing weekly Omega of ETF :  MXI
Computing weekly Omega of ETF :  PYZ
Computing weekly Omega of ETF :  SLX
Computing weekly Omega of ETF :  RTM
Computing weekly Omega of ETF :  UYM
Computing weekly Omega of ETF :  SMN
Computing weekly Omega of ETF :  FXZ
Computing weekly Omega of ETF :  MOO
Computing weekly Omega of ETF :  CUT
Computing weekly Omega of ETF :  WOOD
Computing weekly Omega of ETF :  HAP
Computing weekly Omega of ETF :  PSAU
Computing weekly Omega of ETF :  PAGG
Computing weekly Omega of ETF :  FLM
Computing weekly Omega of ETF :  GRES
Computing weekly Omega of ETF :  GDXJ
Computing weekly Omega of ETF :  CHIM
Computing weekly Omega of ETF : 

In [83]:
#2011 JBF omega performance messure ...
#we need Sharpe_omega(L) which equals to omega_x(L) - 1
#F(.) is the cdf of the asset return X defined on the interval (a,b)
#and omega_x(L) = ( integrate from L to b : (1-F(x))dx) / ( integrate from a to L : F(x)dx )
#where L is the return threshold selected by the investor 
#for monthly:
mOmega = []
for now, symbol in enumerate(symbol_list):
    mOmega.append([])
    List = mr[symbol]
    #print(List.shape[0]) #length of the series
    l = List.shape[0]
    for i in range(l-12*3):
            
        L =  mUSG3M["Price"][i+12*3] #risk-free interest rate 
        
        if(not ( float("-inf") <List[i] < float("inf") )): #nan
            mOmega[now].append(float('nan'))
            continue
        X = []
        for j in range(i, i+12*3):
            X.append(List[j])
        X.sort()
        a = X[0]
        b = X[len(X)-1] 
        left_part = 0
        right_part = 0
        for i in range(len(X)):
            if(X[i] < L):
                left_part +=  L - X[i]
            else :
                right_part += X[i] - L
        omega = right_part / left_part
        mOmega[now].append(omega)
print("Monthly omega has been computed")

Monthly omega has been computed


In [14]:
# time range 
wtime = []
for i in range(len(weekly_ETF.index)-52*3) :
    wtime.append(weekly_ETF.index[i+52*3])
mtime = []
for i in range(len(monthly_ETF.index)-12*3) :
    mtime.append(monthly_ETF.index[i+12*3])

In [85]:
#print(len(symbol_list))
#print(len(mOmega))
#arranging the data above(in pandas.dataframe)
mOmega_dic = {}
mOmega_dic.update({'Date' : mtime})
for i in range(len(symbol_list)):
    mOmega_dic.update({symbol_list[i] : mOmega[i]})

mOmega_df = pd.DataFrame.from_dict(mOmega_dic)
mOmega_df.set_index('Date', inplace=True)
#print(mOmega_df)

wOmega_dic = {}
wOmega_dic.update({ "Date" : wtime})
for i in range(len(symbol_list)):
    wOmega_dic.update({symbol_list[i] : wOmega[i]})
wOmega_df = pd.DataFrame.from_dict(wOmega_dic)
wOmega_df.set_index('Date', inplace=True)
#print(wOmega_df)

mASKSR_dic = {}
mASKSR_dic.update({ "Date" : mtime})
for i in range(len(symbol_list)):
    mASKSR_dic.update({symbol_list[i] : mASKSR[i]})
    
mASKSR_df = pd.DataFrame.from_dict(mASKSR_dic)
mASKSR_df.set_index('Date', inplace=True)
#print(mASKSR_df)

wASKSR_dic = {}

wASKSR_dic.update({ "Date" : wtime})
for i in range(len(symbol_list)):
    wASKSR_dic.update({symbol_list[i] : wASKSR[i]})
wASKSR_df = pd.DataFrame.from_dict(wASKSR_dic)
wASKSR_df.set_index('Date', inplace=True)
#print(wASKSR_df)

In [98]:
# ranking the weekly omega
Ranking_wOmega = []
reverse_ranking = []
for i in range(len(wtime)):
    Ranking_wOmega.append([])
print(Ranking_wOmega)
print(len(Ranking_wOmega))

for i in range(len(wtime)):
    #print(i)
    tmp = []
    nan_num = 0
    for j in range(len(symbol_list)):
        tmp.append(wOmega[j][i])
    rank = ss.rankdata(tmp)
    for j in range(len(symbol_list)):
        Ranking_wOmega[i].append(int ( len(symbol_list) - rank[j] + 1) )
    print("Ranking:",Ranking_wOmega[i])
    count = 0
    for j in range(len(symbol_list)):
        if(not ( float("-inf") < wOmega[j][i] < float("inf") )) : #nan
            Ranking_wOmega[i][j] = float("nan")
            count += 1
    # if nan exists, reranking the data
    for j in range(len(symbol_list)):
        if( float("-inf") < wOmega[j][i] < float("inf") ) : #nan
            Ranking_wOmega[i][j] -= count
    #print(Ranking_wOmega[i])

Trans_Ranking = []
for i in range(len(symbol_list)):
    Trans_Ranking.append([])
    for j in range(len(wtime)):
            Trans_Ranking[i].append( Ranking_wOmega[j][i])
    print(Trans_Ranking[i])
    
wOmega_ranking_dic = {}
wOmega_ranking_dic.update({ "Date" : wtime})
for i in range(len(symbol_list)):
    wOmega_ranking_dic.update({symbol_list[i] : Trans_Ranking[i]})
wOmega_ranking_df = pd.DataFrame.from_dict(wOmega_ranking_dic)
wOmega_ranking_df.set_index('Date', inplace=True)
print(wOmega_ranking_df)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
105
Ranking: [66, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 17, 2, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 33, 34, 35, 36, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 18, 1]
Ranking: [24, 33, 59, 22, 10, 16, 13, 38, 50, 45, 26, 47, 15, 34, 58, 20, 39, 25, 17, 51, 41, 57, 32, 54, 35, 19, 63, 66, 61, 37, 36, 49, 30, 53, 65, 18, 62, 44, 60, 11, 56, 52, 55, 40, 31, 46, 27, 48, 64, 23, 4, 5, 1, 6, 7, 2, 8, 21, 43, 29, 14, 42, 12, 3, 9, 28]
Ranking: [28, 36, 59, 23, 12, 19, 13, 37, 50,

In [99]:
# ranking the monthly omega
Ranking_mOmega = []
reverse_ranking = []
for i in range(len(mtime)):
    Ranking_mOmega.append([])
print(Ranking_mOmega)
print(len(Ranking_mOmega))

for i in range(len(mtime)):
    #print(i)
    tmp = []
    nan_num = 0
    for j in range(len(symbol_list)):
        tmp.append(mOmega[j][i])
    rank = ss.rankdata(tmp)
    for j in range(len(symbol_list)):
        Ranking_mOmega[i].append(int ( len(symbol_list) - rank[j] + 1) )
    print("Ranking:",Ranking_mOmega[i])
    count = 0
    for j in range(len(symbol_list)):
        if(not ( float("-inf") < mOmega[j][i] < float("inf") )) : #nan
            Ranking_mOmega[i][j] = float("nan")
            count += 1
    # if nan exists, reranking the data
    for j in range(len(symbol_list)):
        if( float("-inf") < mOmega[j][i] < float("inf") ) : #nan
            Ranking_mOmega[i][j] -= count
    #print(Ranking_mOmega[i])

Trans_Ranking = []
for i in range(len(symbol_list)):
    Trans_Ranking.append([])
    for j in range(len(mtime)):
            Trans_Ranking[i].append( Ranking_mOmega[j][i])
    print(Trans_Ranking[i])
    
mOmega_ranking_dic = {}
mOmega_ranking_dic.update({ "Date" : mtime})
for i in range(len(symbol_list)):
    mOmega_ranking_dic.update({symbol_list[i] : Trans_Ranking[i]})
mOmega_ranking_df = pd.DataFrame.from_dict(mOmega_ranking_dic)
mOmega_ranking_df.set_index('Date', inplace=True)
print(mOmega_ranking_df)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
24
Ranking: [66, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 17, 2, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 33, 34, 35, 36, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 18, 1]
Ranking: [28, 35, 61, 25, 13, 21, 14, 34, 48, 45, 30, 47, 18, 36, 58, 24, 38, 23, 17, 54, 37, 60, 43, 55, 33, 20, 65, 66, 59, 40, 32, 49, 31, 51, 64, 15, 63, 42, 57, 10, 56, 53, 52, 39, 29, 46, 19, 44, 62, 26, 4, 5, 1, 6, 7, 8, 2, 16, 50, 22, 11, 41, 12, 3, 9, 27]
Ranking: [25, 34, 63, 23, 12, 20, 14, 39, 49, 44, 28, 43, 18, 32, 57, 22, 31, 17, 16, 56, 40, 60, 33, 54, 35, 15, 65, 66, 59, 37, 38, 48, 30, 53, 64, 21, 62, 45, 50, 9, 58, 55, 52, 41, 36, 46, 29, 47, 61, 24, 3, 4, 1, 5, 6, 7, 2, 19, 51, 26, 11, 42, 13, 8, 10, 27]
Ranking: [23, 30, 64, 21, 11, 15, 8, 40, 50, 38, 26, 44, 17, 31, 62, 18, 24, 16, 14, 54, 41, 58, 28, 53, 

In [101]:
# ranking the weekly ASKSR
Ranking_wASKSR = []
reverse_ranking = []
for i in range(len(wtime)):
    Ranking_wASKSR.append([])
print(Ranking_wASKSR)
print(len(Ranking_wASKSR))

for i in range(len(wtime)):
    #print(i)
    tmp = []
    nan_num = 0
    for j in range(len(symbol_list)):
        tmp.append(wASKSR[j][i])
    rank = ss.rankdata(tmp)
    for j in range(len(symbol_list)):
        Ranking_wASKSR[i].append(int ( len(symbol_list) - rank[j] + 1) )
    print("Ranking:",Ranking_wASKSR[i])
    count = 0
    for j in range(len(symbol_list)):
        if(not ( float("-inf") < wASKSR[j][i] < float("inf") )) : #nan
            Ranking_wASKSR[i][j] = float("nan")
            count += 1
    # if nan exists, reranking the data
    for j in range(len(symbol_list)):
        if( float("-inf") < wASKSR[j][i] < float("inf") ) : #nan
            Ranking_wASKSR[i][j] -= count
    #print(Ranking_wOmega[i])

Trans_Ranking = []
for i in range(len(symbol_list)):
    Trans_Ranking.append([])
    for j in range(len(wtime)):
            Trans_Ranking[i].append( Ranking_wASKSR[j][i])
    print(Trans_Ranking[i])
    
wASKSR_ranking_dic = {}
wASKSR_ranking_dic.update({ "Date" : wtime})
for i in range(len(symbol_list)):
    wASKSR_ranking_dic.update({symbol_list[i] : Trans_Ranking[i]})
wASKSR_ranking_df = pd.DataFrame.from_dict(wASKSR_ranking_dic)
wASKSR_ranking_df.set_index('Date', inplace=True)
print(wASKSR_ranking_df)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
105
Ranking: [66, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 17, 2, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 33, 34, 35, 36, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 18, 1]
Ranking: [22, 25, 62, 26, 63, 15, 24, 61, 13, 64, 14, 21, 36, 43, 27, 42, 18, 41, 48, 66, 59, 32, 55, 31, 47, 60, 46, 44, 33, 34, 17, 29, 40, 52, 35, 39, 9, 51, 45, 50, 37, 53, 23, 65, 56, 54, 11, 49, 38, 30, 4, 5, 6, 7, 2, 1, 8, 10, 58, 19, 12, 16, 28, 3, 20, 57]
Ranking: [23, 25, 61, 26, 65, 15, 24, 62, 13,

In [102]:
# ranking the monthly ASKSR
Ranking_mASKSR = []
reverse_ranking = []
for i in range(len(mtime)):
    Ranking_mASKSR.append([])
print(Ranking_mASKSR)
print(len(Ranking_mASKSR))

for i in range(len(mtime)):
    #print(i)
    tmp = []
    nan_num = 0
    for j in range(len(symbol_list)):
        tmp.append(mASKSR[j][i])
    rank = ss.rankdata(tmp)
    for j in range(len(symbol_list)):
        Ranking_mASKSR[i].append(int ( len(symbol_list) - rank[j] + 1) )
    print("Ranking:",Ranking_mASKSR[i])
    count = 0
    for j in range(len(symbol_list)):
        if(not ( float("-inf") < mASKSR[j][i] < float("inf") )) : #nan
            Ranking_mASKSR[i][j] = float("nan")
            count += 1
    # if nan exists, reranking the data
    for j in range(len(symbol_list)):
        if( float("-inf") < mASKSR[j][i] < float("inf") ) : #nan
            Ranking_mASKSR[i][j] -= count
    #print(Ranking_mOmega[i])

Trans_Ranking = []
for i in range(len(symbol_list)):
    Trans_Ranking.append([])
    for j in range(len(mtime)):
            Trans_Ranking[i].append( Ranking_mASKSR[j][i])
    print(Trans_Ranking[i])
    
mASKSR_ranking_dic = {}
mASKSR_ranking_dic.update({ "Date" : mtime})
for i in range(len(symbol_list)):
    mASKSR_ranking_dic.update({symbol_list[i] : Trans_Ranking[i]})
mASKSR_ranking_df = pd.DataFrame.from_dict(mASKSR_ranking_dic)
mASKSR_ranking_df.set_index('Date', inplace=True)
print(mASKSR_ranking_df)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
24
Ranking: [66, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 17, 2, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 33, 34, 35, 36, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 18, 1]
Ranking: [38, 18, 47, 22, 11, 43, 45, 40, 52, 59, 48, 56, 31, 15, 63, 42, 24, 9, 26, 46, 36, 20, 57, 64, 25, 65, 14, 34, 19, 21, 27, 58, 62, 53, 66, 16, 12, 44, 51, 13, 54, 49, 61, 41, 33, 10, 37, 35, 32, 17, 4, 5, 6, 7, 2, 1, 8, 23, 50, 28, 29, 39, 30, 3, 55, 60]
Ranking: [41, 19, 40, 22, 17, 45, 49, 36, 51, 54, 48, 52, 26, 15, 59, 43, 28, 9, 27, 34, 31, 20, 58, 62, 14, 65, 11, 39, 18, 25, 21, 55, 66, 50, 63, 24, 12, 44, 46, 16, 53, 42, 60, 37, 30, 10, 38, 33, 23, 13, 3, 1, 4, 5, 6, 2, 7, 29, 47, 32, 35, 61, 64, 8, 56, 57]
Ranking: [54, 13, 42, 10, 28, 38, 48, 34, 52, 62, 51, 55, 21, 8, 46, 41, 25, 16, 18, 50, 29, 23, 64, 66, 